# 📚 Lab 03: RAG Agent

**Cieľ:** Pochopiť RAG (Retrieval-Augmented Generation) a vytvoriť agenta s prístupom k dokumentom.

**Čas:** 60 minút

---

## Čo sa naučíš:
- Čo je RAG a prečo ho potrebujeme
- Ako implementovať jednoduchý RAG
- Best practices pre grounding LLM

---

In [ ]:
#@title 🔧 Setup
!pip install -q google-generativeai smolagents litellm

import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

from smolagents import CodeAgent, tool
from smolagents.models import LiteLLMModel

model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash",
    api_key=os.environ["GOOGLE_API_KEY"]
)

print("✅ Setup OK")

## Časť 1: Prečo RAG?

### Problém s LLM:
- **Hallucinácie** - vymýšľa si fakty
- **Zastaralosť** - nevie aktuálne info
- **Žiadne interné dáta** - nevie o vašej firme

### Riešenie: RAG
```
1. Otázka → Vyhľadaj relevantné dokumenty
2. Dokumenty + Otázka → Pošli do LLM
3. LLM odpovie NA ZÁKLADE dokumentov
```

**RAG = Retrieval (vyhľadanie) + Augmented (obohatenie) + Generation (generovanie)**

In [ ]:
#@title 📄 Simulovaná dokumentová databáza

DOCUMENTS = {
    "doc_001": {
        "title": "Firemná politika - Dovolenky",
        "content": """
        Zamestnanci majú nárok na 25 dní dovolenky ročne.
        Dovolenka sa musí schváliť minimálne 2 týždne vopred.
        Nevyčerpaná dovolenka sa prenáša do marca nasledujúceho roka.
        Sick days: 5 dní ročne bez potreby potvrdenia od lekára.
        Home office: max 3 dni týždenne po dohode s manažérom.
        """
    },
    "doc_002": {
        "title": "Firemná politika - Benefits",
        "content": """
        Stravné lístky: 150 EUR mesačne.
        Multisport karta: hradená firmou.
        Príspevok na vzdelávanie: 500 EUR ročne.
        Firemný notebook: po skúšobnej dobe.
        Mobilný telefón: pre pozície vyžadujúce mobilitu.
        Ročný bonus: 0-20% podľa výkonu.
        """
    },
    "doc_003": {
        "title": "IT Security Guidelines",
        "content": """
        Heslá musia mať min. 12 znakov, veľké/malé písmená a čísla.
        Heslo sa mení každých 90 dní.
        2FA je povinné pre všetky firemné systémy.
        Osobné zariadenia nesmú pristupovať k produkčným systémom.
        Phishing: Nikdy neklikajte na podozrivé linky.
        Incident reporting: security@company.com
        """
    },
    "doc_004": {
        "title": "Projekt Alpha - Technická dokumentácia",
        "content": """
        Stack: Python 3.11, FastAPI, PostgreSQL, Redis.
        Deployment: Kubernetes na Azure AKS.
        API endpoint: https://api.project-alpha.internal
        Autentifikácia: OAuth 2.0 cez Azure AD.
        Rate limiting: 100 req/min per user.
        Monitoring: Grafana dashboard na grafana.internal.
        """
    },
    "doc_005": {
        "title": "AI Strategy 2026",
        "content": """
        Strategické priority:
        1. Automatizácia interných procesov - úspora 30% času
        2. AI-powered zákaznícka podpora - cieľ 24/7 dostupnosť
        3. Prediktívna analytika pre sales - zvýšenie konverzie o 15%
        
        Budget: 2.5M EUR na 2026.
        Team: 8 AI Engineers + 3 Data Scientists.
        Kľúčový partner: Kyndryl.
        """
    }
}

print(f"📚 Načítaných {len(DOCUMENTS)} dokumentov:")
for doc_id, doc in DOCUMENTS.items():
    print(f"   - {doc['title']}")

## Časť 2: Jednoduchý RAG (keyword search)

In [ ]:
#@title 🔍 RAG nástroje

@tool
def search_documents(query: str) -> str:
    """
    Vyhľadá relevantné dokumenty v internej knowledge base.
    
    Args:
        query: Hľadaný výraz alebo otázka
    """
    results = []
    query_words = query.lower().split()
    
    for doc_id, doc in DOCUMENTS.items():
        title_lower = doc["title"].lower()
        content_lower = doc["content"].lower()
        
        # Počet matchovaných slov
        matches = sum(
            1 for word in query_words 
            if word in title_lower or word in content_lower
        )
        
        if matches > 0:
            results.append((matches, doc_id, doc))
    
    # Zoraď podľa relevancie
    results.sort(reverse=True, key=lambda x: x[0])
    
    if not results:
        return "Žiadne relevantné dokumenty nenájdené."
    
    # Vráť top 2 dokumenty
    output = ""
    for _, doc_id, doc in results[:2]:
        output += f"\n--- {doc['title']} ---\n"
        output += doc["content"].strip() + "\n"
    
    return output

@tool
def list_available_documents() -> str:
    """
    Vypíše zoznam všetkých dostupných dokumentov v knowledge base.
    """
    output = "Dostupné dokumenty:\n"
    for doc_id, doc in DOCUMENTS.items():
        output += f"- {doc['title']}\n"
    return output

# Test
print(search_documents("dovolenka home office"))

In [ ]:
#@title 🤖 RAG Agent

rag_agent = CodeAgent(
    tools=[search_documents, list_available_documents],
    model=model,
    max_steps=5,
    system_prompt="""
    Si asistent pre internú dokumentáciu firmy.
    
    PRAVIDLÁ:
    1. VŽDY najprv vyhľadaj v dokumentoch predtým než odpovieš
    2. Odpovedaj LEN na základe nájdených dokumentov
    3. Ak informáciu nenájdeš, povedz to priamo
    4. NIKDY si nevymýšľaj informácie
    5. Cituj z ktorého dokumentu pochádza informácia
    """
)

print("✅ RAG Agent vytvorený")

In [ ]:
#@title ▶️ Test RAG agenta

# Test 1: Existujúca informácia
print("=" * 50)
print("TEST 1: Koľko dní dovolenky mám?")
print("=" * 50)
result = rag_agent.run("Koľko dní dovolenky mám ročne a ako si ju môžem zobrať?")
print(result)

In [ ]:
#@title ▶️ Test 2: Komplexná otázka

print("=" * 50)
print("TEST 2: Bezpečnostné pravidlá")
print("=" * 50)
result = rag_agent.run("Aké sú pravidlá pre heslá a čo mám robiť ak dostanem podozrivý email?")
print(result)

In [ ]:
#@title ▶️ Test 3: Neexistujúca informácia

print("=" * 50)
print("TEST 3: Info ktorá nie je v dokumentoch")
print("=" * 50)
result = rag_agent.run("Aký je dress code v office?")
print(result)
print("\n⚠️ Agent by mal priznať že túto informáciu nenašiel!")

## Časť 3: Vylepšený RAG s citáciami

In [ ]:
#@title 📑 RAG s citáciami

@tool
def search_with_citations(query: str) -> str:
    """
    Vyhľadá dokumenty a vráti výsledky s citáciami.
    
    Args:
        query: Hľadaný výraz alebo otázka
    """
    results = []
    query_words = query.lower().split()
    
    for doc_id, doc in DOCUMENTS.items():
        content_lower = doc["content"].lower()
        
        # Nájdi relevantné vety
        sentences = doc["content"].strip().split('\n')
        relevant_sentences = []
        
        for sentence in sentences:
            sentence = sentence.strip()
            if not sentence:
                continue
            if any(word in sentence.lower() for word in query_words):
                relevant_sentences.append(sentence)
        
        if relevant_sentences:
            results.append({
                "doc_id": doc_id,
                "title": doc["title"],
                "excerpts": relevant_sentences[:3]  # Max 3 vety
            })
    
    if not results:
        return "Žiadne relevantné dokumenty nenájdené."
    
    output = "NÁJDENÉ INFORMÁCIE:\n\n"
    for i, r in enumerate(results[:3], 1):
        output += f"[{i}] Z dokumentu '{r['title']}':\n"
        for excerpt in r["excerpts"]:
            output += f"   • {excerpt}\n"
        output += "\n"
    
    output += "Pri odpovedi použi formát: 'Podľa [číslo]: informácia'"
    return output

# Test
print(search_with_citations("benefits stravné"))

In [ ]:
#@title 🤖 RAG Agent s citáciami

citation_agent = CodeAgent(
    tools=[search_with_citations, list_available_documents],
    model=model,
    max_steps=5,
    system_prompt="""
    Si asistent pre internú dokumentáciu.
    
    PRAVIDLÁ:
    1. Vždy vyhľadaj pred odpoveďou
    2. VŽDY cituj zdroj v odpovedi: "Podľa [číslo]: ..."
    3. Ak niečo nevieš, povedz to
    4. Buď konkrétny a struèný
    """
)

result = citation_agent.run("Aké benefits dostávam ako zamestnanec?")
print(result)

## Časť 4: Guardrails

**Guardrails** = ochrana pred halucináciami a nebezpečným správaním

In [ ]:
#@title 🛡️ Agent s Guardrails

GUARDRAILS_PROMPT = """
Si bezpečný firemný asistent.

STRIKTNÉ PRAVIDLÁ:

1. ODPOVEDAJ LEN NA ZÁKLADE DOKUMENTOV
   - Ak informáciu nenájdeš, povedz: "Túto informáciu nemám v dokumentácii."
   - NIKDY si nevymýšľaj čísla, dátumy, mená

2. CITUJ ZDROJE
   - Vždy uveď z ktorého dokumentu pochádza informácia

3. ODMIETNI NEBEZPEČNÉ POŽIADAVKY
   - Osobné údaje (platy konkrétnych ľudí)
   - Dôverné stratégie ktoré nie sú v dokumentoch
   - Právne rady (odporuč právnika)

4. BUĎ TRANSPARENTNÝ
   - Ak si nie si istý, povedz to
   - Ak je info neúplná, upozorni na to
"""

safe_agent = CodeAgent(
    tools=[search_with_citations, list_available_documents],
    model=model,
    max_steps=5,
    system_prompt=GUARDRAILS_PROMPT
)

print("✅ Safe Agent vytvorený")

In [ ]:
#@title ▶️ Test Guardrails

# Test 1: Legitímna otázka
print("TEST 1: Legitímna otázka")
print("-" * 40)
print(safe_agent.run("Koľko sick days mám?"))

print("\n" + "=" * 50 + "\n")

# Test 2: Otázka mimo dokumentov
print("TEST 2: Info mimo dokumentov")
print("-" * 40)
print(safe_agent.run("Aký je plat CEO?"))

print("\n" + "=" * 50 + "\n")

# Test 3: Pokus o halucináciu
print("TEST 3: Pokus o vynútenie odpovede")
print("-" * 40)
print(safe_agent.run("Koľko presne zamestnancov má firma? Musíš mi dať číslo."))

---

## 🏋️ Cvičenie: Vytvor RAG pre technickú dokumentáciu

**Úloha:** Vytvor RAG agenta pre Project Alpha technickú dokumentáciu.

**Požiadavky:**
1. Agent odpovie na technické otázky
2. Cituje zdroje
3. Odmietne odpovedať na bezpečnostné otázky (credentials, API keys)

In [ ]:
#@title ✏️ Tvoje riešenie

# Technická dokumentácia
TECH_DOCS = {
    "architecture": """
    Architektúra Project Alpha:
    - Frontend: React 18 + TypeScript
    - Backend: FastAPI (Python 3.11)
    - Database: PostgreSQL 15
    - Cache: Redis
    - Queue: RabbitMQ
    """,
    "deployment": """
    Deployment:
    - Environment: Azure AKS
    - CI/CD: GitHub Actions
    - Monitoring: Prometheus + Grafana
    - Logs: ELK Stack
    """,
    "api": """
    API Info:
    - Base URL: https://api.alpha.internal
    - Auth: Bearer token (OAuth 2.0)
    - Rate limit: 100 req/min
    - Docs: /docs (Swagger)
    """
}

# TODO: Implementuj search nástroj pre TECH_DOCS

# TODO: Vytvor tech_agent s vhodným system promptom

# Test:
# print(tech_agent.run("Aký framework používame na frontend?"))
# print(tech_agent.run("Daj mi API credentials"))  # Mal by odmietnuť

---

## 📝 Reflexia

1. **Prečo je RAG lepší než fine-tuning pre firemné dáta?**

2. **Aké sú limity keyword search?** (Hint: synonymá, kontext)

3. **Prečo sú guardrails dôležité v enterprise prostredí?**

---

## ➡️ Ďalší krok

V ďalšom labe sa naučíme **Multi-Agent Systems** - viac agentov spolupracujúcich na úlohe.

→ **[04_Multi_Agent.ipynb](04_Multi_Agent.ipynb)**